### Investment and Trading Capstone Project
# Build a Stock Price Indicator

## Description

Investment firms, hedge funds and even individuals have been using financial models to better understand market behavior and make profitable investments and trades. A wealth of information is available in the form of historical stock prices and company performance data, suitable for machine learning algorithms to process.


For this project, the task is to build a stock price predictor that takes daily trading data over a certain date range as input, and outputs projected estimates for given query dates. Note that the inputs will contain multiple metrics, such as opening price (Open), highest price the stock traded at (High), how many stocks were traded (Volume) and closing price adjusted for stock splits and dividends (Adjusted Close); your system only needs to predict the Adjusted Close price.


Feel free to choose what form your project takes (a simple script, a web app/service, Android/iOS app, etc.), and any additions/modifications to make to the project (e.g. suggesting what trades to make). Make sure you document your intended features in your report.

### Step one: Download the data

1.Yahoo! Finance: You can directly query for a stock through the web API, or download a dump of .csv files and use them.

2.Bloomberg API: Multiple APIs available, including Python.

3.Quandl: Also multiple APIs, including Python.

#### Input:
Open, High, Low, Close, Volume and Adjusted Close
#### Output:
Adjusted Close

#### Yahoo Finance:
Direct Download: Go to Yahoo Finance.
-Enter a quote into the search field.
-Select a quote in the search results to view it.
-Click Historical Data.
-Select a Time Period, data to Show, and Frequency.
-Click Apply.
-To use the data offline, click Download Data.

In [2]:
import numpy as np
import pandas as pd

In [6]:
### example of .csv file downloaded from Yahoo Finance
fdyh1 = pd.read_csv('./^GSPC.csv')
fdyh1.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
247,2019-08-14,2894.149902,2894.149902,2839.639893,2840.600098,2840.600098,4312530000
248,2019-08-15,2846.199951,2856.669922,2825.510010,2847.600098,2847.600098,4038000000
249,2019-08-16,2864.739990,2893.629883,2864.739990,2888.679932,2888.679932,3498150000
250,2019-08-19,2913.479980,2931.000000,2913.479980,2923.649902,2923.649902,3212880000
251,2019-08-20,2919.010010,2923.629883,2903.959961,2908.370117,2908.370117,1161061878


In [ ]:
### Rapid API for yahoo finance
import unirest
response = unirest.post(API_URL,
  headers={
    "X-RapidAPI-Key": API_KEY,
    "Content-Type": "application/x-www-form-urlencoded"
  },
  params={
    "parameter": "value"
  }
)
response.code # The HTTP status code
response.headers # The HTTP headers
response.body # The parsed response
response.raw_body # The unparsed response

ModuleNotFoundError: No module named 'pandas_datareader'